In [1]:
!pip install fastapi uvicorn pyngrok transformers torch langchain langchain-community langchain-huggingface sentence-transformers faiss-cpu  pyngrok rank-bm25 underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━

# Config.py

In [2]:
import os
from pathlib import Path

CONFIG = {
    "index_path": Path("/kaggle/input/qcdt-data-ver-2/index.faiss"),
    "docs_path": Path("/kaggle/input/qcdt-data-ver-2/docs.pkl"),
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "reranker_model": "BAAI/bge-reranker-base",
    "default_dense_k": 10,
    "default_bm25_k": 10,
    "default_top_n": 3
}

# Query_system.py

In [3]:
import os
import pickle
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from underthesea import word_tokenize
from langchain_core.documents import Document
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

class QuestionClassifier:
    def __init__(self, model_path="/kaggle/input/query-model-ver-1/transformers/default/1/question_classifier"):
        print(f"Đang tải mô hình phân loại từ: {model_path}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()
        self.id2label = self.model.config.id2label

    def classify(self, question: str) -> str:
        inputs = self.tokenizer(
            question, 
            return_tensors="pt", 
            truncation=True, 
            padding=True,
            max_length=128
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        prediction = torch.argmax(outputs.logits, dim=1).item()
        return self.id2label[prediction]

class AdvancedQuerySystem:
    def __init__(self, config):
        self.config = CONFIG
        self.classifier = QuestionClassifier()
        print("Đang khởi tạo retrieval")
        self.embedding_model = SentenceTransformer(self.config['embedding_model'])
        self.reranker = CrossEncoder(self.config['reranker_model'])
        self.index = faiss.read_index(str(self.config['index_path']))
        with open(self.config['docs_path'], "rb") as f:
            self.docs = pickle.load(f)
        self.corpus = [doc['content'] for doc in self.docs]
        tokenized_corpus = [word_tokenize(doc) for doc in self.corpus]
        self.bm25 = BM25Okapi(tokenized_corpus)
        self.content_to_doc_map = {doc['content']: doc for doc in self.docs}
        print("Ready")
        
    def _dense_retrieval(self, question: str, k: int) -> list:
        query_vector = self.embedding_model.encode(question)
        query_vector_2d = np.array([query_vector], dtype='float32')
        distances, indices = self.index.search(query_vector_2d, k=k)
        return [self.docs[i] for i in indices[0]]

    def _bm25_retrieval(self, question: str, k: int) -> list:
        tokenized_query = word_tokenize(question)
        top_k_contents = self.bm25.get_top_n(tokenized_query, self.corpus, n=k)
        return [self.content_to_doc_map[content] for content in top_k_contents]

    def _rerank(self, question: str, candidate_docs: list[Document], top_n: int) -> list[Document]:        
        reranker_input = [[question, doc.page_content] for doc in candidate_docs]
        scores = self.reranker.predict(reranker_input)
        doc_score_pairs = list(zip(candidate_docs, scores))
        doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
        reranked_docs = [doc for doc, score in doc_score_pairs[:top_n]]
        return reranked_docs

    def query(self, question: str) -> list[Document]:                
        question_type = self.classifier.classify(question)
        print(f"Loại câu hỏi được xác định: {question_type}")
        
        # Chiến lược truy vấn
        dense_k = self.config['default_dense_k']
        bm25_k = self.config['default_bm25_k']
        top_n = self.config['default_top_n']

        if question_type == "Definition" or question_type == "Factoid":            
            dense_k = 8
            bm25_k = 12  # Ưu tiên BM25
            top_n = 1    # Chỉ cần 1 chunk là đủ
        elif question_type == "List":            
            dense_k = 12
            bm25_k = 12
            top_n = 5    # Lấy nhiều chunk hơn chút vì cần list chunk ra                        
        elif question_type == "Inference":            
            dense_k = 15  # Tăng mạnh k để suy luận tốt hơn
            bm25_k = 15
            top_n = 5    
        # Loại Y/N thì để tham số mặc định

        # HYBRID SEARCH        
        dense_results = self._dense_retrieval(question, k=dense_k)
        bm25_results = self._bm25_retrieval(question, k=bm25_k)
        
        all_results_dict = {doc_data['content']: doc_data for doc_data in dense_results + bm25_results}
        candidate_docs_data = list(all_results_dict.values())
        
        candidate_docs_lc = [
            Document(page_content=doc['content'], metadata=doc['metadata']) 
            for doc in candidate_docs_data
        ]        

        # RERANKING
        final_docs = self._rerank(question, candidate_docs_lc, top_n)
        
        return final_docs

2025-07-31 19:03:01.021168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753988581.372828      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753988581.471332      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model.py

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from functools import lru_cache
from peft import PeftModel
import os
import torch
import re

MODEL_ID_MAP = {
    "Qwen3 4B pretrain": "/kaggle/input/qwen3-4b-legal-pretrain/transformers/default/1/qwen3-4b-legal-pretain",
    "Qwen3 1.7B": "/kaggle/input/qwen-3/transformers/1.7b/1",
    "Qwen3 4B finetune": "/kaggle/input/qwen3-4b-finetune-ver3/transformers/default/1/kaggle/working/qwen_vietnamese_qa",
}

@lru_cache(maxsize=3)
def get_model_and_tokenizer(model_name):
    model_id = MODEL_ID_MAP.get(model_name, None)
    if model_id is None:
        raise ValueError(f"Mô hình '{model_name}' không được hỗ trợ. Các mô hình hợp lệ: {list(MODEL_ID_MAP.keys())}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    return model, tokenizer

def generate_answer(question, context="", model_name=None):
    model, tokenizer = get_model_and_tokenizer(model_name)
    
    prompt = f"""Bạn là trợ lý AI trả lời câu hỏi về quy chế đào tạo của trường Đại học Công nghệ, Đại học Quốc gia Hà Nội. Dựa trên ngữ cảnh dưới đây, trả lời câu hỏi một cách ngắn gọn và chính xác. Nếu ngữ cảnh không cung cấp đủ thông tin, hãy trả lời rằng thông tin không có sẵn và không suy đoán. Không tự sinh thêm câu hỏi, chỉ sinh câu trả lời và chỉ sinh một câu trả lời duy nhất.

Ngữ cảnh: {context}

Câu hỏi: {question}

Trả lời: """
    print("\n--- PROMPT (input to model) ---\n" + prompt)
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=4,
        temperature=0.3,  
        top_p=0.5        
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n--- RAW OUTPUT (from model) ---\n" + answer)

    if "Trả lời:" in answer:
        answer = answer.split("Trả lời:", 1)[-1].strip()
    else:
        answer = answer[len(prompt):].strip()

    answer = re.sub(r'(<think>|</think>|<\|thinking\|>|<\|/thinking\|>|Quyết định này|Bản quyền ©.*$)', '', answer, flags=re.DOTALL)
    answer = re.sub(r'BULLET \d+.*?(?=(BULLET \d+|$))', '', answer, flags=re.DOTALL)
    answer = re.sub(r'[ \t]+', ' ', answer).strip()
    answer = re.sub(r'\n\s*\n+', '\n', answer).strip()

    sentences = re.split(r'(?<=[.!?])\s+', answer.strip())
    if sentences and not re.search(r'[.!?]$', sentences[-1]):
        sentences = sentences[:-1]  # Loại bỏ câu cuối không hoàn chỉnh
    answer = ' '.join(sentences).strip()
    
    print("\n--- FINAL ANSWER (postprocessed) ---\n" + answer)

    questions_in_answer = re.findall(r"[^\n.?!]*\?", answer)
    if questions_in_answer:
        print("\n--- QUESTIONS FOUND IN ANSWER ---")
        for q in questions_in_answer:
            print(q.strip())
    else:
        print("\n--- NO QUESTIONS FOUND IN ANSWER ---")
    
    return answer
 

# Main.py

In [5]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class QuestionRequest(BaseModel):
    question: str
    context: str = ""
    model: str = ""

class AnswerResponse(BaseModel):
    answer: str

query_system = AdvancedQuerySystem(CONFIG)

@app.post("/ask", response_model=AnswerResponse)
async def ask(request: QuestionRequest):
    retrieved_docs = query_system.query(request.question)

    context = request.context + "\n\nThông tin liên quan từ tài liệu:\n"
    for i, doc in enumerate(retrieved_docs):
        context += f"Chunk {i+1}: {doc.page_content}\n\n"
        print(f"Chunk {i+1}: {doc.page_content[:]}...")  
    answer = generate_answer(request.question, context, model_name=request.model)
    return {"answer": answer}

Đang tải mô hình phân loại từ: /kaggle/input/query-model-ver-1/transformers/default/1/question_classifier
Đang khởi tạo retrieval


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Ready


30UmaHTtKzQUSIV45GgrcLODwX1_7A3Ummwv3yLCvZxkCtzCi

In [67]:
ngrok.kill()


In [6]:
from pyngrok import ngrok
ngrok.set_auth_token('30UmaHTtKzQUSIV45GgrcLODwX1_7A3Ummwv3yLCvZxkCtzCi')  # Thay bằng token Ngrok của bạn

# Dừng tất cả tunnel hiện có
ngrok.kill()

import uvicorn
import nest_asyncio
import threading

nest_asyncio.apply()

# Chạy server FastAPI trong luồng nền
def run_server():
    uvicorn.run(app, host='0.0.0.0', port=8000)

server_thread = threading.Thread(target=run_server)
server_thread.start()

# Tạo URL công khai với Ngrok
public_url = ngrok.connect(8000, bind_tls=True)
print("Public URL:", public_url)

INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://5a29d34287d9.ngrok-free.app" -> "http://localhost:8000"
INFO:     2402:9d80:26e:7112:1199:8f4b:cdfb:e37:0 - "OPTIONS /ask HTTP/1.1" 200 OK
Loại câu hỏi được xác định: List


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk 1: 1. Học phân là một phần kiến thức của chương trình đảo tạo; mỗi học phần
có khối lượng kiến thức từ 2 đến 5 tín chỉ, được tổ chức giảng dạy trọn vẹn trong
một học kỳ (trừ học phân thực tập thực tế); mỗi học phần có mã số riêng do thủ
trưởng đơn vị đào tạo ban hành dựa trên nguyên tắc đánh mã của Đại học Quốc gia
Hà Nội.
2. Các loại học phần
a) Học phần bắt buộc: sinh viên bắt buộc phải tích lũy;
b) Học phần tự chọn có điều kiện: sinh viên được tự chọn theo hướng dẫn
của đơn vị đào tạo;
c) Học phần tự chọn tự do: sinh viên tự chọn theo nhu cầu cá nhân phù hợp
các yêu câu học tập;
d) Học phần tiên quyết của một học phần: sinh viên phải tích lũy trước khi
học học phần đó;
đ) Khóa luận, đồ án tốt nghiệp là học phần thuộc khối kiến thức bắt buộc, có
thời lượng từ 5 đến 10 tín chỉ được quy định cụ thể trong chương trình đào tạo;
e) Thực tập, thực tế (nếu có) là một học phần bắt buộc;
g) Học phần điều kiện là các học phần giáo dục thể chất, giáo dục quốc
phòng - an ninh và kỹ năng b

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk 1: l. Nội dung đào tạo, phương pháp dạy - học, kiểm tra đánh giá và phương
thức quản lý phải phù hợp và đáp ứng chuẩn đầu ra của chương trình đào tạo.
2. Phát triển các chương trình đảo tạo mới, có tính liên ngành đáp ứng nhu
cầu hiện tại và tương lai của xã hội.
3. Ưu tiên đầu tư điều kiện đảm bảo chất lượng giáo dục.
4. Gắn đào tạo với nghiên cứu khoa học, hoạt động thực tiễn.
5. Kiểm định chất lượng giáo dục là yêu cầu bắt buộc trong đào tạo....

--- PROMPT (input to model) ---
Bạn là trợ lý AI trả lời câu hỏi về quy chế đào tạo của trường Đại học Công nghệ, Đại học Quốc gia Hà Nội. Dựa trên ngữ cảnh dưới đây, trả lời câu hỏi một cách ngắn gọn và chính xác. Nếu ngữ cảnh không cung cấp đủ thông tin, hãy trả lời rằng thông tin không có sẵn và không suy đoán. Không tự sinh thêm câu hỏi, chỉ sinh câu trả lời và chỉ sinh một câu trả lời duy nhất.

Ngữ cảnh: 

Thông tin liên quan từ tài liệu:
Chunk 1: l. Nội dung đào tạo, phương pháp dạy - học, kiểm tra đánh giá và phương
thức quản

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk 1: l. Nội dung đào tạo, phương pháp dạy - học, kiểm tra đánh giá và phương
thức quản lý phải phù hợp và đáp ứng chuẩn đầu ra của chương trình đào tạo.
2. Phát triển các chương trình đảo tạo mới, có tính liên ngành đáp ứng nhu
cầu hiện tại và tương lai của xã hội.
3. Ưu tiên đầu tư điều kiện đảm bảo chất lượng giáo dục.
4. Gắn đào tạo với nghiên cứu khoa học, hoạt động thực tiễn.
5. Kiểm định chất lượng giáo dục là yêu cầu bắt buộc trong đào tạo....

--- PROMPT (input to model) ---
Bạn là trợ lý AI trả lời câu hỏi về quy chế đào tạo của trường Đại học Công nghệ, Đại học Quốc gia Hà Nội. Dựa trên ngữ cảnh dưới đây, trả lời câu hỏi một cách ngắn gọn và chính xác. Nếu ngữ cảnh không cung cấp đủ thông tin, hãy trả lời rằng thông tin không có sẵn và không suy đoán. Không tự sinh thêm câu hỏi, chỉ sinh câu trả lời và chỉ sinh một câu trả lời duy nhất.

Ngữ cảnh: 

Thông tin liên quan từ tài liệu:
Chunk 1: l. Nội dung đào tạo, phương pháp dạy - học, kiểm tra đánh giá và phương
thức quản